**Building a RAG Pipeline for LLM**

In [5]:
%pip install -U PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00


In [11]:
%pip install -U langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00


In [6]:
!pip install langchain chromadb sentence-transformers huggingface_hub pypdf requests beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 11.7 MB/s eta

In [7]:
# Upload your `sss_links.txt` file in Colab
from google.colab import files
uploaded = files.upload()

with open("resource.txt", "r") as f:
    urls = [line.strip() for line in f.readlines() if line.strip()]

Saving resource.txt to resource.txt


In [8]:
import requests
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader
import io

def fetch_html(url):
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.text, "html.parser")
        return soup.get_text()
    except Exception as e:
        return ""

def fetch_pdf(url):
    try:
        res = requests.get(url)
        reader = PdfReader(io.BytesIO(res.content))
        return "\n".join([page.extract_text() or "" for page in reader.pages])
    except Exception as e:
        return ""


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)

documents = []

for url in urls:
    print(f"Fetching: {url}")
    text = fetch_pdf(url) if url.endswith(".pdf") else fetch_html(url)
    if text:
        chunks = text_splitter.split_text(text)
        for chunk in chunks:
            documents.append(Document(page_content=chunk, metadata={"source": url}))


Fetching: https://www.sss.gov.ph/
Fetching: https://www.sss.gov.ph/the-sss-mandate/
Fetching: https://www.sss.gov.ph/ss-commission/
Fetching: https://www.sss.gov.ph/management-directory/
Fetching: https://www.sss.gov.ph/wp-content/uploads/2025/06/SSS-Citizens-Charter-2025-1st-Edition.pdf
Fetching: https://www.sss.gov.ph/wp-content/uploads/2022/05/GCG_SSS_AR15.pdf
Fetching: https://www.sss.gov.ph/wp-content/uploads/2022/05/GCG_SSS_AR16.pdf
Fetching: https://www.sss.gov.ph/wp-content/uploads/2022/05/GCG_SSS_AR17.pdf
Fetching: https://www.sss.gov.ph/wp-content/uploads/2022/05/GCG_SSS_AR18.pdf
Fetching: https://www.sss.gov.ph/wp-content/uploads/2022/05/GCG_SSS_AR19.pdf
Fetching: https://www.sss.gov.ph/wp-content/uploads/2023/02/SSS-Annual-Report_FINAL_2021.pdf
Fetching: https://www.sss.gov.ph/wp-content/uploads/2024/10/2022-SSS-Annual-Report-Final.pdf
Fetching: https://www.sss.gov.ph/wp-content/uploads/2025/01/SSS-2023-Annual-Report-full.pdf
Fetching: https://www.sss.gov.ph/mysss-pension-b

In [ ]:
import os
# Replace this

# 2. Import and create the embedding model
from langchain.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
)

/tmp/ipython-input-12-193676579.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(documents, embedding, persist_directory="sss_chroma")
db.persist()

/tmp/ipython-input-13-4215310586.py:4: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [14]:
!pip install transformers accelerate


In [15]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.llms import HuggingFacePipeline

model_id = "google/flan-t5-small"  # Very lightweight and good for CPU

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Create text2text-generation pipeline
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=False
)

# Wrap it for LangChain
llm = HuggingFacePipeline(pipeline=pipe)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu
/tmp/ipython-input-15-1742689772.py:20: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [16]:
from langchain.vectorstores import Chroma

db = Chroma(persist_directory="sss_chroma", embedding_function=embedding)

/tmp/ipython-input-16-1722415258.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory="sss_chroma", embedding_function=embedding)


In [17]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=db.as_retriever(),
    return_source_documents=True
)

In [18]:
query = "How do I apply for an SSS salary loan?"
response = qa_chain({"query": query})

print("✅ Answer:\n", response["result"])

print("\n📚 Sources:")
for doc in response["source_documents"]:
    print("-", doc.metadata["source"])

/tmp/ipython-input-18-807955232.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"query": query})
Token indices sequence length is longer than the specified maximum sequence length for this model (806 > 512). Running this sequence through the model will result in indexing errors


✅ Answer:
 Apply Online Definition Qualifying Conditions Loan Details Loan Proceeds Responsibilities of Employer (for employed members) Responsibilities of Member-Borrower How to Apply Definition It is a privilege loan granted to eligible member-borrowers intended to meet their short-term credit needs. Qualifying Conditions All employed and currently contributing self-employed or voluntary members applying for salary loan must meet the following eligibility requirements: Who may avail:All employed SSS members must have met the following requirements: 1. For a one-month salary loan, the member-borrower must have thirty-six (36) posted monthly contributions, six (6) of which should be within the last twelve (12) months prior to the month of filing of application. 2. For a two-month salary loan, the member-borrower must have seventy-two (72) posted monthly contributions, six (6) of which should be within the last twelve (12) months prior to the month of filing of application. 3. The emplo